In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from scipy import interp


from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

In [ ]:
def calibration(data, train_pop, target_pop, sampled_train_pop, sampled_target_pop):

    calibrated_data = \
    ((data * (target_pop / train_pop) / (sampled_target_pop / sampled_train_pop)) /
    ((
        (1 - data) * (1 - target_pop / train_pop) / (1 - sampled_target_pop / sampled_train_pop)
     ) +
     (
        data * (target_pop / train_pop) / (sampled_target_pop / sampled_train_pop)
     )))

    return calibrated_data

In [ ]:
qs_post = pd.read_csv("qs_2016_2017_prediction_mortality_211213.csv", index_col=0)

qs_post_2016 = qs_post[qs_post["dtjahr_a"]==2016].copy()
qs_post_2016 = qs_post_2016.drop(columns=["dtjahr_a",])
qs_post_2017 = qs_post[qs_post["dtjahr_a"]==2017].copy()
qs_post_2017 = qs_post_2017.drop(columns=["dtjahr_a",])

In [ ]:
no_downsamples = 100
inner_cv = 5

feature_importances = []


all_preds_2016 = []
all_y_2016 = []
all_preds_2017 = []
all_y_2017 = []

tprs_2016 = []
tprs_2017 = []
test_auc_2016 = []
test_auc_2017 = []


fraction_pos_all_2017 = []
mean_pred_all_2017 = []
brier_score_all = []

fraction_pos_all_2017_cal = []
mean_pred_all_2017_cal = []
test_auc_2017_cal = []

mean_fpr = np.linspace(0, 1, 100)

for no in range(no_downsamples):
    print(no)
    
    qs_post_2016_0 = qs_post_2016[qs_post_2016["tot"]==0].copy()
    qs_post_2016_1 = qs_post_2016[qs_post_2016["tot"]==1].copy()

    qs_post_2016_0_sample = qs_post_2016_0.sample(int(np.sum(qs_post_2016["tot"])), replace=False)
    qs_post_2016_1_sample = qs_post_2016_1.sample(int(np.sum(qs_post_2016["tot"])), replace=False)

    qs_post_2016_joint = pd.concat([qs_post_2016_0_sample, qs_post_2016_1_sample])

    y = np.array(qs_post_2016_joint["tot"])
    qs_post_2016_joint = qs_post_2016_joint.drop(["tot"], axis=1)
    
    qs_post_2017_joint = qs_post_2017.copy()

    y_2017 = np.array(qs_post_2017_joint["tot"])
    qs_post_2017_joint = qs_post_2017_joint.drop(["tot"], axis=1)
                
    pipe = Pipeline([
               ('scale', StandardScaler()),
                    ('classify', GradientBoostingClassifier(learning_rate=0.1,
                                       max_features='log2'))         
                                    ])
            
    N_ESTIMATORS = [100, 300, 500]
    MAX_DEPTH = [1,3, 5]
    LOSS = ['deviance', 'exponential']
    
                
    parameters = {'classify__n_estimators': N_ESTIMATORS, 
                  'classify__max_depth': MAX_DEPTH,
                  'classify__loss': LOSS}
    
    X_train, X_test, y_train, y_test = train_test_split(qs_post_2016_joint, y, test_size=0.2, random_state=no)

    clf = GridSearchCV(pipe,param_grid=parameters, cv=inner_cv)
    
    clf.fit(X_train, y_train)
    
    preds = clf.predict(X_test)
    preds_2017 = clf.predict(qs_post_2017_joint)
    all_preds_2016.append(preds)
    all_y_2016.append(y_test)
    all_preds_2017.append(preds_2017)
    all_y_2017.append(np.array(y_2017).ravel())
        
    y_proba_2016 = pd.DataFrame(clf.predict_proba(X_test)).loc[:,1]
            
    fpr_2016, tpr_2016, thresholds_2016 = roc_curve(y_test, y_proba_2016)
    tprs_2016.append(np.interp(mean_fpr, fpr_2016, tpr_2016))
    test_auc_2016.append(roc_auc_score(y_test, y_proba_2016))
        
    y_proba_2017 = pd.DataFrame(clf.predict_proba(qs_post_2017_joint)).loc[:,1]
            
    fpr_2017, tpr_2017, thresholds_2017 = roc_curve(y_2017, y_proba_2017)
    tprs_2017.append(np.interp(mean_fpr, fpr_2017, tpr_2017))
    test_auc_2017.append(roc_auc_score(y_2017, y_proba_2017))
        
    fraction_of_positives, mean_predicted_value = calibration_curve(y_2017, y_proba_2017, n_bins=10)
    brier_score = brier_score_loss(y_2017, y_proba_2017, pos_label=y_2017.max())
    brier_score_all.append(brier_score)
    fraction_pos_all_2017.append(fraction_of_positives)
    mean_pred_all_2017.append(mean_predicted_value)
        
    calibrated_y_proba_2017 = calibration(y_proba_2017, len(y_2017), np.sum(y_2017), len(y), np.sum(y))
    fraction_of_positives_cal, mean_predicted_value_cal = calibration_curve(y_2017, calibrated_y_proba_2017, n_bins=100,strategy='quantile')
    fraction_pos_all_2017_cal.append(fraction_of_positives_cal)
    mean_pred_all_2017_cal.append(mean_predicted_value_cal)
    test_auc_2017_cal.append(roc_auc_score(y_2017, calibrated_y_proba_2017))
    

    feature_importances = np.append(feature_importances, clf.best_estimator_.named_steps["classify"].feature_importances_)

In [ ]:
# Saving prediction scores

scores = pd.DataFrame(test_auc_2016)
scores.columns = ["validation_auc_2016"]
scores["auc_2017"] = test_auc_2017
scores["auc_cal_2017"] = test_auc_2017_cal
scores["brier_2017"] = brier_score_all

scores.to_csv("predictions/GBC_mortality_noICU_1111.csv")

tprs_long = pd.DataFrame(tprs_2017)
tprs_long.to_csv("predictions/GBC_mortality_tprs_noICU_1111.csv")

fraction_pos_all_2017 = pd.DataFrame(fraction_pos_all_2017_cal)
fraction_pos_all_2017.to_csv("predictions/GBC_mortality_fraction_pos_cal_noICU_1111.csv")

mean_pred_all_2017 = pd.DataFrame(mean_pred_all_2017_cal)
mean_pred_all_2017.to_csv("predictions/GBC_mortality_mean_pred_cal_noICU_1111.csv")

pd.DataFrame(all_preds_2016).to_csv("predictions/GBC/GBC_mortality_all_pred_2016_noICU_1111.csv")
pd.DataFrame(all_preds_2017).to_csv("predictions/GBC/GBC_mortality_all_pred_2017_noICU_1111.csv")
pd.DataFrame(all_y_2016).to_csv("predictions/GBC/GBC_mortality_all_y_2016_noICU_1111.csv")
pd.DataFrame(all_y_2017).to_csv("predictions/GBC/GBC_mortality_all_y_2017_noICU_1111.csv")

overall_feature_importances = pd.DataFrame(np.reshape(feature_importances, (100, -1))).mean(axis=0)
overall_feature_importances.index = qs_post_2016_joint.columns
overall_feature_importances = overall_feature_importances.sort_values()
overall_feature_importances.to_csv("predictions/GBC_mortality_fi_noICU_1111.csv")